In [2]:
import os

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
if torch.cuda.is_available():
    dev = "cuda:0"
    num_workers = 1
    pin_memory = True
else:
    dev = "cpu"
    num_workers = 0
    pin_memory = False
num_workers = 0
pin_memory = False

In [ ]:
root_dir = '../slices'

In [ ]:
slice_df = pd.read_csv("../slice_filenames.csv")
slice_df.head()

In [ ]:
class SignalDataset(Dataset):
    def __init__(self, slice_df):
        self.slice_df = slice_df

    def __len__(self):
        return len(self.slice_df)

    def __getitem__(self, idx):
        row = self.slice_df.iloc[idx,:]
        filename = row['filename']
        fold = row['fold']
        x = np.load(os.path.join(root_dir, f"fold{fold}", filename))
        x = torch.tensor(x, device=dev).float().unsqueeze(0)
        y = torch.tensor(row['classID'], device=dev)
        return x, y

In [ ]:
train_df = slice_df[(slice_df['fold'] != 8) & (slice_df['fold'] != 9)]
test_df = slice_df[(slice_df['fold'] == 8) | (slice_df['fold'] == 9)]

In [ ]:
root_dir = '../slices'

In [ ]:
slice_df = pd.read_csv("../slice_filenames.csv")
slice_df.head()

In [ ]:
class SignalDataset(Dataset):
    def __init__(self, slice_df):
        self.slice_df = slice_df

    def __len__(self):
        return len(self.slice_df)

    def __getitem__(self, idx):
        row = self.slice_df.iloc[idx,:]
        filename = row['filename']
        fold = row['fold']
        x = np.load(os.path.join(root_dir, f"fold{fold}", filename))
        x = torch.tensor(x, device=dev).float().unsqueeze(0)
        y = torch.tensor(row['classID'], device=dev)
        return x, y

In [56]:
train_df = slice_df[(slice_df['fold'] != 8) & (slice_df['fold'] != 9)]
test_df = slice_df[(slice_df['fold'] == 8) | (slice_df['fold'] == 9)]

In [64]:
class SignalModel(nn.Module):
    def __init__(self, n_channels=32):
        super(SignalModel, self).__init__()
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim=2)
        self.conv1 = nn.Conv1d(1, n_channels, kernel_size=240, stride=16)
        self.bn1 = nn.BatchNorm1d(n_channels)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channels, n_channels, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channels)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channels, 2 * n_channels, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channels)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channels, 2 * n_channels, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channels)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channels, 10)
        """self.layer = nn.Sequential( #44100
            nn.Conv1d(in_channels=1, out_channels=n_channels, kernel_size=300, padding=150, stride=2, bias=False), #22050
            nn.BatchNorm1d(n_channels),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=2, padding=1), #11025
            nn.Conv1d(in_channels=n_channels, out_channels=n_channels, kernel_size=3, stride=2, padding=1), #5513
            nn.BatchNorm1d(n_channels),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=2, padding=1), #2757
            nn.Conv1d(in_channels=n_channels, out_channels=n_channels, kernel_size=3, stride=2, padding=1), #1379
            nn.BatchNorm1d(n_channels),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=2, padding=1), #690
            nn.Conv1d(in_channels=n_channels, out_channels=n_channels, kernel_size=3, stride=2, padding=1), #345
            nn.BatchNorm1d(n_channels),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=2, padding=1), #173
            nn.Flatten(),
            nn.Linear(in_features=173*n_channels, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10),
        )"""

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = self.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return torch.squeeze(x)

In [65]:
model = SignalModel().to(device=dev)

In [66]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
train_loop(train_dl, test_dl, model, loss_fn, optimizer)
for _ in range(3):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
    train_loop(train_dl, test_dl, model, loss_fn, optimizer)
for _ in range(6):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.001)
    train_loop(train_dl, test_dl, model, loss_fn, optimizer)

  0%|          | 0/120 [00:00<?, ?it/s]

train_loss: 2.171638990441958, val_loss: 2.1370315296309337


  0%|          | 0/120 [00:00<?, ?it/s]

train_loss: 2.082924457391103, val_loss: 2.0913774285997664


  0%|          | 0/120 [00:00<?, ?it/s]

...

train_loss: 0.9032951680322489, val_loss: 1.6375148126057215


  0%|          | 0/120 [00:00<?, ?it/s]

train_loss: 0.8972707418724895, val_loss: 1.640453195997647


  0%|          | 0/120 [00:00<?, ?it/s]

train_loss: 0.8902540589372318, val_loss: 1.640840313264302


In [62]:
one_pass_acc(model, test_dl)

torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([256, 10])
torch.Size([230, 10])


0.4567348081769812